In [2]:
import pandas as pd
import random
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
from sentence_transformers import CrossEncoder
import numpy as np

In [3]:
ds_train = load_dataset("FreedomIntelligence/RAG-Instruct", split="train[:80%]")
ds_test = load_dataset("FreedomIntelligence/RAG-Instruct", split="train[80%:]")

README.md:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

rag_instruct.json:   0%|          | 0.00/296M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40541 [00:00<?, ? examples/s]

In [4]:
docs_train = [d for s in ds_train['documents'] for d in s]
questions_train = [q for q in ds_train['question']]

docs_test = [d for s in ds_test['documents'] for d in s]
questions_test = [q for q in ds_test['question']]

#### Sentence Transformer

In [5]:
st_model = 'multi-qa-mpnet-base-dot-v1'

model = SentenceTransformer(st_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
docs_embeddings_train = model.encode(docs_train, batch_size=64, convert_to_tensor=True)

docs_embeddings_test = model.encode(docs_test, batch_size=64, convert_to_tensor=True)

Batches:   0%|          | 0/5068 [00:00<?, ?it/s]

Batches:   0%|          | 0/1267 [00:00<?, ?it/s]

In [7]:
questions_embeddings_train = model.encode(questions_train, batch_size=64, convert_to_tensor=True)

questions_embeddings_test = model.encode(questions_test, batch_size=64, convert_to_tensor=True)

Batches:   0%|          | 0/507 [00:00<?, ?it/s]

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

In [8]:
docs_embeddings_train = docs_embeddings_train.cpu().numpy()
questions_embeddings_train = questions_embeddings_train.cpu().numpy()

docs_embeddings_test = docs_embeddings_test.cpu().numpy()
questions_embeddings_test = questions_embeddings_test.cpu().numpy()

In [9]:
np.save(f"/kaggle/working/st_docs_embeddings_train-{st_model}.npy", docs_embeddings_train)
np.save(f"/kaggle/working/st_docs_embeddings_test-{st_model}.npy", docs_embeddings_test)

In [10]:
np.save(f"/kaggle/working/st_qsts_embeddings_train-{st_model}.npy", questions_embeddings_train)
np.save(f"/kaggle/working/st_qsts_embeddings_test-{st_model}.npy", questions_embeddings_test)

#### Cross encoder

In [23]:
xenc_model = 'stsb-distilroberta-base'

xmodel = CrossEncoder(f'cross-encoder/{xenc_model}')

In [12]:
def build_xenc_input(questions, docs):
    xinputs = []
    for i in range(len(questions)):
        for j in range(10):
            xinputs.append((questions[i], docs[i * 10 + j]))
    return xinputs

In [14]:
xinputs_train = build_xenc_input(questions_train, docs_train)
xinputs_test = build_xenc_input(questions_test, docs_test)

In [30]:
def xenc_predict(xinputs, questions):
    scores = xmodel.predict(xinputs)
    grouped_scores = [scores[i*10:(i+1)*10] for i in range(len(questions))]
    per_q_scores = []
    for i in range(len(grouped_scores)):
        scores = grouped_scores[i]
        score_index_pairs = [(score, j) for j, score in enumerate(scores)]
        score_index_pairs.sort(reverse=True)
        per_q_scores.append(score_index_pairs)
    return per_q_scores

In [31]:
per_q_scores_train = xenc_predict(xinputs_train, questions_train)
per_q_scores_test = xenc_predict(xinputs_test, questions_test)

Batches:   0%|          | 0/10136 [00:00<?, ?it/s]

Batches:   0%|          | 0/2534 [00:00<?, ?it/s]

In [33]:
np.save(f"/kaggle/working/xenc_scores_train-{xenc_model}.npy", per_q_scores_train)
np.save(f"/kaggle/working/xenc_scores_test-{xenc_model}.npy", per_q_scores_test)

In [35]:
import shutil

shutil.make_archive('/kaggle/working/my_archive', 'zip', '/kaggle/working')

'/kaggle/working/my_archive.zip'